In [1]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 4.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from tigramite import plotting as tp
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
def get_nonlinear_equations(n_vars, max_lag):
    """Get nonlinear equations with confounder U"""
    if n_vars == 4:
        if max_lag == 2:
            return [
                "X4[t] = 0.25 * X1[t-2]^2 - 0.1 * X1[t-2]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.3 * U[t]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X4[t] = 0.25 * X1[t-2]^2 - 0.1 * X1[t-2]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.2 * X2[t-3]^2 + 0.3 * U[t]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X4[t] = 0.25 * X1[t-4]^2 - 0.1 * X1[t-4]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.2 * X2[t-3]^2 + 0.3 * U[t]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
    elif n_vars == 6:
        base_equations = get_nonlinear_equations(4, max_lag)
        additional = [
            "X6[t] = 0.45 * X5[t]^2 - 0.15 * X5[t]^3 + 0.4 * U[t]^2 + e6",
            "X5[t] = 0.3 * X4[t-1]^2 - 0.1 * X4[t-1]^3 + e5"
        ]
        return additional + base_equations
    elif n_vars == 8:
        base_equations = get_nonlinear_equations(6, max_lag)
        additional = [
            "X8[t] = 0.4 * X7[t]^2 - 0.12 * X7[t]^3 + 0.35 * U[t]^2 + e8",
            "X7[t] = 0.35 * X6[t-1]^2 - 0.08 * X6[t-1]^3 + e7"
        ]
        return additional + base_equations
    return []

def extract_linear_links(equations):
    """Extract all linear and nonlinear causal links from the equations"""
    links = {}

    for eq in equations:
        if '=' in eq:
            left, right = [side.strip() for side in eq.split('=')]
            target = left.split('[')[0]

            if target != 'U':  # Skip U's equation
                terms = [term.strip() for term in right.split('+')]
                for term in terms:
                    if '*' in term and ('X' in term or 'U' in term):
                        # Extract coefficient and variable
                        coeff = float(term.split('*')[0].strip())
                        var_part = term.split('*')[1].strip()

                        # Handle power terms by taking only the base variable
                        var = var_part.split('^')[0].split('[')[0]
                        lag_part = var_part.split('[')[1].split(']')[0]

                        # Get lag
                        lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])

                        # Store the link with its base coefficient
                        key = (var, lag, target)
                        if key not in links:
                            links[key] = 0
                        links[key] += coeff  # Sum coefficients if multiple terms

    return links

class NonlinearConfoundedGenerator:
    def __init__(self, error_dist='gaussian', noise_params={'scale': 0.1, 'df': 3}, random_state=None):
        self.error_dist = error_dist
        self.noise_params = noise_params
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)

    def generate_noise(self, size):
        if self.error_dist == 'gaussian':
            return np.random.normal(0, self.noise_params['scale'], size=size)
        elif self.error_dist == 't':
            return stats.t.rvs(df=self.noise_params['df'],
                             scale=self.noise_params['scale'],
                             size=size,
                             random_state=self.random_state)

    def generate_equations(self, t, X, U, n_vars, max_lag):
        noise = self.generate_noise(n_vars + 1)  # +1 for U
        equations = get_nonlinear_equations(n_vars, max_lag)
        true_links = extract_coefficients_from_equations(equations)

        # Generate U first as it's a confounder
        U[t] = noise[-1]

        # Generate values based on nonlinear relationships
        for i in range(n_vars-1, -1, -1):
            var_name = f'X{i+1}'
            value = 0

            # Add causal influences with polynomial terms
            for (source, lag, target, power), coef in true_links.items():
                if target == var_name:
                    if source == 'U':
                        value += coef * (U[t] ** power)
                    else:
                        source_idx = int(source[1:]) - 1
                        if lag == 0:
                            value += coef * (X[t, source_idx] ** power)
                        else:
                            value += coef * (X[t + lag, source_idx] ** power)

            # Add noise term
            X[t, i] = value + noise[i]

    def generate_multivariate_ts(self, n_points, n_vars, max_lag):
        # Initialize arrays
        X = np.zeros((n_points, n_vars))
        U = np.zeros(n_points)  # Array for confounder U

        # Initialize first steps with noise
        for i in range(max_lag):
            X[i] = self.generate_noise(n_vars)
            U[i] = self.generate_noise(1)[0]

        # Generate time series
        for t in range(max_lag, n_points):
            self.generate_equations(t, X, U, n_vars, max_lag)

        # Create DataFrame
        columns = [f'X{i+1}' for i in range(n_vars)] + ['U']
        df = pd.DataFrame(np.column_stack([X, U[:, np.newaxis]]), columns=columns)
        df['time'] = np.arange(n_points)

        return df

def extract_coefficients_from_equations(equations):
    """Extract coefficients and causal relationships from equations"""
    causal_links = {}

    for eq in equations:
        if '=' not in eq:
            continue

        left, right = [side.strip() for side in eq.split('=')]
        if 'e' in right and len(right.split('+')) == 1:
            continue  # Skip pure noise equations

        target = left.split('[')[0]
        terms = [term.strip() for term in right.split('+')]

        for term in terms:
            if '*' not in term or not ('X' in term or 'U' in term):
                continue

            # Split coefficient and variable part
            parts = term.split('*')
            coeff = float(parts[0].strip())
            var_part = parts[1].strip()

            # Extract variable name and power
            base_var = var_part.split('^')[0] if '^' in var_part else var_part
            var = base_var.split('[')[0]

            # Get power
            power = 1
            if '^' in var_part:
                power = int(var_part.split('^')[1].split(' ')[0])

            # Get lag
            if var == 'U':
                lag = 0
            else:
                lag_part = base_var.split('[')[1].split(']')[0]
                lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])

            causal_links[(var, lag, target, power)] = coeff

    return causal_links

def save_dataset_and_graph(df, n_vars, max_lag, sample_size, error_dist, output_dir="output"):
    """Save dataset and create causal graph"""
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    base_filename = f'{output_dir}/nonlinear_confounded_n{sample_size}_vars{n_vars}_lag{max_lag}_{error_dist}'

    # Save dataset
    df.to_csv(f'{base_filename}.csv', index=False)

    # Get equations and extract links
    equations = get_nonlinear_equations(n_vars, max_lag)
    true_links = extract_linear_links(equations)

    # Create variable names including U
    var_names = [f'X{i+1}' for i in range(n_vars)] + ['U']
    n_total_vars = n_vars + 1
    val_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1))
    graph_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1), dtype='bool')

    # Fill matrices based on true links
    for (source, lag, target), weight in true_links.items():
        if source == 'U':
            source_idx = n_vars
        else:
            source_idx = int(source[1:]) - 1

        target_idx = int(target[1:]) - 1
        lag_idx = abs(lag)

        # Add the link to the matrices
        val_matrix[source_idx, target_idx, lag_idx] = weight
        graph_matrix[source_idx, target_idx, lag_idx] = True

        # For contemporaneous links, make val_matrix symmetric
        if lag == 0:
            val_matrix[target_idx, source_idx, lag_idx] = weight

    # Plot and save causal graph
    plt.figure(figsize=(12, 12))
    tp.plot_time_series_graph(
        val_matrix=val_matrix,
        graph=graph_matrix,
        var_names=var_names,
        link_colorbar_label='Nonlinear Effect Strength',
        node_size=0.05
    )
    plt.title(f'Nonlinear Causal Graph (n={sample_size}, vars={n_vars}, lag={max_lag}, with confounder)')
    plt.savefig(f'{base_filename}_graph.png')
    plt.close()

    # Plot time series
    plt.figure(figsize=(15, 10))
    for col in df.columns[:-2]:  # Exclude U and time
        plt.plot(df['time'], df[col], label=col, alpha=0.7)
    plt.title(f'Time Series (n={sample_size}, vars={n_vars}, lag={max_lag})')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{base_filename}_series.png')
    plt.close()

    # Save causal structure
    with open(f'{base_filename}_structure.txt', 'w') as f:
        f.write("True Nonlinear Causal Structure with Confounder:\n")
        f.write("Format: (source, lag, target) => weight\n")
        for link, weight in true_links.items():
            f.write(f"{link} => {weight}\n")

def analyze_generated_data(df, max_lag, title="Nonlinear Time Series Analysis"):
    """Analyze and plot the generated time series data"""
    print(f"\nAnalyzing {title}")
    print("=" * 50)

    # Time Series Plot
    plt.figure(figsize=(15, 10))
    for col in df.columns:
        if col != 'time':
            plt.plot(df['time'], df[col], label=col, alpha=0.7)
    plt.title(f"{title} - Time Series")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Summary Statistics
    print("\nSummary Statistics:")
    print("-" * 30)
    print(df.drop('time', axis=1).describe())

    # Correlation Analysis
    print("\nCorrelation with Confounder U:")
    print("-" * 30)
    u_corr = df.drop('time', axis=1).corr()['U'].sort_values(ascending=False)
    print(u_corr)

if __name__ == "__main__":
    # # Example analysis of a single case
    # print("Generating example dataset with Gaussian noise...")

    # # Parameters for example
    # n_points = 1000
    # n_vars = 4
    # max_lag = 2

    # # Generate Gaussian example
    # generator = NonlinearConfoundedGenerator(
    #     error_dist='gaussian',
    #     noise_params={'scale': 0.1},
    #     random_state=42
    # )

    # df_gaussian = generator.generate_multivariate_ts(
    #     n_points=n_points,
    #     n_vars=n_vars,
    #     max_lag=max_lag
    # )

    # # Save and analyze Gaussian example
    # save_dataset_and_graph(
    #     df=df_gaussian,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     sample_size=n_points,
    #     error_dist='gaussian'
    # )

    # analyze_generated_data(df_gaussian, max_lag, "Gaussian Example")

    # # Generate Student's t example
    # print("\nGenerating example dataset with Student's t noise...")
    # generator_t = NonlinearConfoundedGenerator(
    #     error_dist='t',
    #     noise_params={'scale': 0.1, 'df': 3},
    #     random_state=42
    # )

    # df_t = generator_t.generate_multivariate_ts(
    #     n_points=n_points,
    #     n_vars=n_vars,
    #     max_lag=max_lag
    # )

    # # Save and analyze t-distribution example
    # save_dataset_and_graph(
    #     df=df_t,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     sample_size=n_points,
    #     error_dist='t'
    # )

    # analyze_generated_data(df_t, max_lag, "Student's t Example")

    # Generate all combinations
    def generate_all_combinations():
        """Generate datasets for all combinations"""
        sample_sizes = [500, 1000, 3000, 5000]
        n_vars_list = [4, 6, 8]
        max_lags = [2, 3, 4]
        error_dists = ['gaussian', 't']

        for n in sample_sizes:
            for vars in n_vars_list:
                for lag in max_lags:
                    for dist in error_dists:
                        print(f"\nGenerating dataset: n={n}, vars={vars}, lag={lag}, dist={dist}")

                        noise_params = {'scale': 0.1, 'df': 3} if dist == 't' else {'scale': 0.1}
                        generator = NonlinearConfoundedGenerator(
                            error_dist=dist,
                            noise_params=noise_params,
                            random_state=42
                        )

                        df = generator.generate_multivariate_ts(
                            n_points=n,
                            n_vars=vars,
                            max_lag=lag
                        )

                        save_dataset_and_graph(df, vars, lag, n, dist)
                        print("Dataset and visualizations saved successfully")

    # Uncomment to generate all combinations
    # print("\nGenerating all combinations...")
    generate_all_combinations()


Generating dataset: n=500, vars=4, lag=2, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=2, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=2, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, dist=t
Dataset and visualizations saved successfully

Gene

/usr/local/lib/python3.11/dist-packages/tigramite/plotting.py:3203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = pyplot.figure(figsize=figsize)
<ipython-input-4-d698c621f5d4>:236: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 10))


Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=3, dist=gaussian


<ipython-input-4-d698c621f5d4>:223: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=3, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=4, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=4, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=2, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=2, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=3, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=3, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=4, dist=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=4, dist=t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=8, lag=2, dist=gaus

...

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [ ]:
# if __name__ == "__main__":
    # # Example analysis of a single case
    # print("Generating example dataset with Gaussian noise...")

    # # Parameters for example
    # n_points = 1000
    # n_vars = 4
    # max_lag = 2

    # # Generate Gaussian example
    # generator = NonlinearConfoundedGenerator(
    #     error_dist='gaussian',
    #     noise_params={'scale': 0.1},
    #     random_state=42
    # )

    # df_gaussian = generator.generate_multivariate_ts(
    #     n_points=n_points,
    #     n_vars=n_vars,
    #     max_lag=max_lag
    # )

    # # Save and analyze Gaussian example
    # save_dataset_and_graph(
    #     df=df_gaussian,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     sample_size=n_points,
    #     error_dist='gaussian'
    # )

    # analyze_generated_data(df_gaussian, max_lag, "Gaussian Example")

    # # Generate Student's t example
    # print("\nGenerating example dataset with Student's t noise...")
    # generator_t = NonlinearConfoundedGenerator(
    #     error_dist='t',
    #     noise_params={'scale': 0.1, 'df': 3},
    #     random_state=42
    # )

    # df_t = generator_t.generate_multivariate_ts(
    #     n_points=n_points,
    #     n_vars=n_vars,
    #     max_lag=max_lag
    # )

    # # Save and analyze t-distribution example
    # save_dataset_and_graph(
    #     df=df_t,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     sample_size=n_points,
    #     error_dist='t'
    # )

    # analyze_generated_data(df_t, max_lag, "Student's t Example")

In [5]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/nonlinear_confounded_n500_vars6_lag2_gaussian_structure.txt (deflated 46%)
  adding: content/output/nonlinear_confounded_n5000_vars4_lag4_gaussian.csv (deflated 54%)
  adding: content/output/nonlinear_confounded_n1000_vars6_lag2_t.csv (deflated 98%)
  adding: content/output/nonlinear_confounded_n3000_vars6_lag2_t_series.png (deflated 31%)
  adding: content/output/nonlinear_confounded_n5000_vars6_lag3_t_series.png (deflated 30%)
  adding: content/output/nonlinear_confounded_n5000_vars6_lag4_t_series.png (deflated 31%)
  adding: content/output/nonlinear_confounded_n500_vars8_lag2_t.csv (deflated 98%)
  adding: content/output/nonlinear_confounded_n500_vars6_lag3_t.csv (deflated 97%)
  adding: content/output/nonlinear_confounded_n5000_vars4_lag3_gaussian_series.png (deflated 3%)
  adding: content/output/nonlinear_confounded_n500_vars8_lag2_gaussian_series.png (deflated 1%)
  adding: content/output/nonlinear_confounded_n5000_var

In [6]:
from google.colab import files
files.download('/content/output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>